In [1]:
from pyzx import *
from fractions import Fraction

In [2]:
class SQW:
    def __init__(self):
        self.qubit_amount = 3
        self.circ = Circuit(qubit_amount=self.qubit_amount)
    def setup(self):
        self.circ.add_gate("NOT",2)
    def add_layer(self, n=1):
        for i in range(n):
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("CNOT",0,1)
            self.circ.add_gate("NOT",0)
            
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            
            self.circ.add_gate("NOT",0)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("CNOT",0,1)

In [3]:
two_step = SQW()
two_step.setup()
two_step.add_layer(2)

In [4]:
g = two_step.circ
print("With syntactic sugar")
draw(g)
print("Without syntactic sugar")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

With syntactic sugar


Without syntactic sugar


Circuit  on 3 qubits with 77 gates.
        32 is the T-count
        45 Cliffords among which
        28 2-qubit gates (28 CNOT, 0 other) and
        8 Hadamard gates.


# Hand picked simplifications
The simplifications below where chosen because of the structure of the diagram above.

- phase_free_simp -> spider_simp followed by a bialg_simp
- id_simp to remove any $n*2\pi$ rotations
- spider_simp to fuse spiders after the id
- id_simp to remove any $n*2\pi$ rotations

In [5]:
def qw_opt(g):
    phase_free_simp(g,quiet=False)
    id_simp(g,quiet=False)
    spider_simp(g, quiet=False)
    id_simp(g,quiet=False)
    #supplementarity_simp(g,quiet=False)
    g.normalize()

In [6]:
opt = g.to_graph()
qw_opt(opt)
draw(opt)

spider_simp: 14. 11. 7. 5. 3.  5 iterations
id_simp: 5. 3.  2 iterations


In [7]:
result = extract_simple(opt)
print(result.stats(),"\nEQUAL: " , g.verify_equality(result))
print("Result:")
draw(result)
print("Original:")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

Circuit  on 3 qubits with 59 gates.
        32 is the T-count
        27 Cliffords among which
        20 2-qubit gates (20 CNOT, 0 other) and
        2 Hadamard gates. 
EQUAL:  True
Result:


Original:


Circuit  on 3 qubits with 77 gates.
        32 is the T-count
        45 Cliffords among which
        28 2-qubit gates (28 CNOT, 0 other) and
        8 Hadamard gates.


# Full Reduce
This simplification is useful to reduce the T-count of a circuit due to the phase gadget optimizations implemented in the algorithm

In [8]:
fr = g.to_graph()
full_reduce(fr, quiet=False)
fr.normalize()
draw(fr)

spider_simp: 14. 11. 7. 5. 3.  5 iterations
id_simp: 5. 3.  2 iterations
pivot_simp: 1.  1 iterations
pivot_gadget_simp: 11. 4. 2. 2. 2. 1.  6 iterations
id_simp: 4.  1 iterations
spider_simp: 1. 1. 1.  3 iterations
gadget_simp: 8.  1 iterations
id_simp: 1.  1 iterations
spider_simp: 1.  1 iterations
lcomp_simp: 6. 3. 3.  3 iterations


In [9]:
fr_opt = extract_circuit(fr.copy())
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)

Circuit  on 3 qubits with 69 gates.
        16 is the T-count
        53 Cliffords among which
        23 2-qubit gates (8 CNOT, 15 other) and
        28 Hadamard gates. 
EQUAL:  True


PyZX tends to perform better with a lot of hadamard gates, although this circuit could use some optimizations

We can remove some hadamards by some colour changes and the total number of gates with a few spider simps and id simps

In [10]:
fr_graph = fr_opt.to_graph()
# Simple simplifications
spider_simp(fr_graph,quiet=False)
id_simp(fr_graph,quiet=False)
to_rg(fr_graph)
#The simps above dont change the graph too much so we can use the extract_simple
fr_opt = extract_simple(fr_graph)
print()
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)

spider_simp: 18. 14. 8. 9. 3. 2. 1. 1.  8 iterations
id_simp: 3.  1 iterations

Circuit  on 3 qubits with 47 gates.
        16 is the T-count
        31 Cliffords among which
        25 2-qubit gates (19 CNOT, 6 other) and
        4 Hadamard gates. 
EQUAL:  True
